In [ ]:
%pip install transformers datasets
%pip install langdetect
%pip install jupyter
%pip install pandas
%pip install pyarrow
%pip install -U sentence-transformers
%pip install lexrank
%pip install nltk

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("./data/merged_df.csv")
df = df.groupby(["CUI"]).agg({"STR": "first", "DEF": lambda x: " ".join(x)})
df.to_csv("data/CUI_STR_DEF.csv")

In [45]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
paraphrases = util.paraphrase_mining(model, df["DEF"], show_progress_bar=True, top_k=7)
# embeddings = model.encode(df["DEF"][0:100], convert_to_tensor=True)

result = []
for paraphrase in paraphrases:
    score, i, j = paraphrase
    result.append([df.index[i],
            df["STR"][i],
            df.index[j],
            df["STR"][j],
            score])
    # print("{} \t\t {} \t\t Score: {:.4f}".format(df["STR"][i], df["STR"][j], score))
result_df = pd.DataFrame(result, columns=["CUI_1", "STR_1", "CUI_2", "STR_2", "SCORE"])
result_df = result_df.sort_values(["CUI_1", "SCORE"], ascending=[True, False])
result_df.to_csv("output/similarity.csv", index=False)

Batches: 100%|██████████| 45/45 [00:56<00:00,  1.25s/it]


In [ ]:
import pandas as pd
similarity_df = pd.read_csv("output/similarity.csv")
similarity_df


def get_similar_diseases(similarity_df, name: str):
    return similarity_df.loc[similarity_df["STR_1"] == name]

get_similar_diseases(similarity_df, "ABETALIPOPROTEINEMIA")